Q: low values for X_rotation. Is X_rotation actually rotation or is it velocity?

In [ ]:
import pandas as pd
import json

# Load the JSON data from a file
with open('log_240517_1hz.json', 'r') as file:
    data = json.load(file)


rows = []
for entry in data:
    timestamp = entry['timestamp']
    msg_timestamp = entry['msg_timestamp']
    for obj in entry['object_list']:
        row = {
            'timestamp': timestamp,
            'msg_timestamp': msg_timestamp,
            'object_id': obj[0],
            'x_position': obj[1],
            'y_position': obj[2],
            'z_position': obj[3],
            'x_rotation': obj[4],
            'y_rotation': obj[5],
            'z_rotation': obj[6],
        }
        rows.append(row)

df = pd.DataFrame(rows)

df # = df[df['object_id'] == 0]

In [ ]:
# df[df['object_id'] == 13772]
df.describe()

In [ ]:
df['object_id'].nunique()

In [ ]:
%matplotlib widget
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
 
x = df['x_position']
y = df['y_position']
z = df['z_position']

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x, y, z, color='green', s=1)
ax.set_title("Simple 3D Scatter Plot")

# Set aspect ratio
max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0

mid_x = (x.max()+x.min()) * 0.5
mid_y = (y.max()+y.min()) * 0.5
mid_z = (z.max()+z.min()) * 0.5

ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)
plt.show()

In [ ]:
import matplotlib.animation as animation
data = pd.DataFrame(rows)
data = data[["timestamp", "object_id", "x_position", "y_position"]]
print(data.describe())
df = pd.DataFrame(data)
fig, ax = plt.subplots()
cmap = plt.cm.get_cmap("tab10")  #  colormap

def init():
    ax.set_xlim(-26, 50)
    ax.set_ylim(-11, 10)
    return []


def update(frame):
    ax.clear()  # Clear the previous frame
    ax.set_xlim(-26, 50)
    ax.set_ylim(-11, 10)

    # current timestamp
    current_data = df[df["timestamp"] == frame]

    # Plot each object with a different color
    for obj_id in current_data["object_id"].unique():
        obj_data = current_data[current_data["object_id"] == obj_id]
        ax.scatter(
            obj_data["x_position"], obj_data["y_position"], color=cmap(int(obj_id) % 10)
        )
    #print(frame)
    ax.legend(loc="upper right")
    return []


timestamps = df["timestamp"].unique()

ani = animation.FuncAnimation(
    fig, update, frames=timestamps, init_func=init, blit=True, repeat=False
)
ani.save("animation.mp4", writer="ffmpeg", fps=10)
# plt.show()